This notebook plots the rest mass density for a blast wave from spectre volume data using yt. Put this notebook in the same directory as your h5 data and then run it.

In [ ]:
import h5py
import numpy as np
import glob
import yt

In [ ]:
file_prefix = 'GrMhd'
field_to_plot = 'RestMassDensity'
output_file_name = "BlastWave_R7MUSCL_RestMassDensity.png"

# Shouldn't have to change anything below here except perhaps plot options

In [ ]:
# Reads element data from an h5 file
def read_element_data_from_h5(file_name):
    file = h5py.File(file_name)
    element_data = file.get("element_data.vol")
    return element_data

# From element data, return a sorted tuple of times and time_ids
def times_from_element_data(element_data):
    time_ids = np.array([x for x in element_data.keys()])
    times = np.array([element_data.get(x).attrs.get("observation_value") for x in time_ids])
    # sort times and time_ids
    sorted_args = np.argsort(times)
    return (times[sorted_args], time_ids[sorted_args])

def grid_dict_from_grid(grid):
    x = grid.get('InertialCoordinates_x')
    y = grid.get('InertialCoordinates_y')
    z = grid.get('InertialCoordinates_z')
    grid_dict = dict(left_edge = [np.min(x), np.min(y), np.min(z)], 
                     right_edge = [np.max(x), np.max(y), np.max(z)],
                     level = 0,
                     dimensions = list(x.shape))
    return grid_dict

def grid_dict_from_grid_and_field(grid, field_name):
    grid_dict = grid_dict_from_grid(grid)
    grid_dict.update({field_name:np.array(grid.get(field_name))})
    return grid_dict

def append_to_plot_data(plot_data, element_data, time_id, field_name):
    grid_keys = np.array([x for x in element_data.get(time_id).keys()])
    for grid_key in grid_keys:
        grid = element_data.get(time_id).get(grid_key)
        plot_data.append(grid_dict_from_grid_and_field(grid, field_name))

In [ ]:
plot_data = []

files = [(h5py.File(filename, 'r'), filename)
            for filename in glob.glob(file_prefix + "*.h5")]
times = times_from_element_data(files[0][0].get("element_data.vol"))

for file in files:
    print(file[-1])
    element_data = file[0].get("element_data.vol")
    append_to_plot_data(plot_data, element_data, times[-1][-1], field_to_plot)

plot_dimensions = plot_data[0].get("dimensions")

In [ ]:
ds = yt.load_amr_grids(plot_data, plot_dimensions)

In [ ]:
plt = yt.SlicePlot(ds, "z", ["RestMassDensity"], True, width=12.0, center=(0,0,0))
plt.set_cmap("RestMassDensity", 'viridis')
plt.annotate_title("Rest Mass Density $(t=4)$")
plt.set_log("RestMassDensity", True)
plt.set_zlim("RestMassDensity", 1.e-4, 1.e-3)
plt.set_xlabel('x')
plt.set_ylabel('y')
plt.set_colorbar_label('RestMassDensity','')
plt.show()
plt.save(output_file_name)